# Model Notebook
Inputs = dataloader objects from Data.py
\
Outputs = model training tensorboard dashboards


## 0. Import

To run with tensorboard.

In [192]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [193]:
#!pip install torch torchvision

In [194]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [195]:
## From Hu et al. (2020)
import pickle
import pandas as pd
import numpy as np
from numpy.random import seed; seed(111)
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from IPython.display import Image

## 0. Import
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F

## 1. Load data

In [196]:
class AdDataset(Dataset):
    def __init__(self, data_dir):
        ## Build a list of tuples
        # Here x is our .csv files (i.e. CyTOF data)
        # Here y is our output (0 for a control and 1 for a AD patient)

        self.data_dir = data_dir
        self.x = os.listdir(data_dir)
        self.y= []  # Initialize an empty list to store class labels

        for file_name in self.x:
            # Extract the letter preceding ".csv" in the file name
            y_label = file_name.split(".")[0][-1]
            # Check if the class label is "C" and assign 0, else assign 1
            if y_label == "C":
                self.y.append(0)
                #print(0)
            else:
                self.y.append(1)
                #print(1)

    def __len__(self):
        ## Size of whole data set
        return len(self.x)

    def __getitem__(self, idx):
        ## For loading data on demand, rather than loaded in __init__ step, to increase memory inefficiency.

        file_path = os.path.join(self.data_dir, self.x[idx])
        data = pd.read_csv(file_path, sep="\t", header=None).values
        data = torch.from_numpy(data)
        label = self.y[idx]  # Get the class label for the corresponding file WATCH OUT FOR FLOAT --> MAY CAUSE ERRORS BECAUSE DATA NOT IN SAME DTYPE AS CLASS_LABEL
        #dimensions = data.shape  # Get the dimensions of the data

        return data, label

In [197]:
data_dir_train = "/content/drive/MyDrive/colabData/st1/train" # 290
train_dataset = AdDataset(data_dir_train)


data_dir_val = "/content/drive/MyDrive/colabData/st1/validate"
val_dataset = AdDataset(data_dir_val)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=4, shuffle=True)

# 2. Model performance

In [198]:
class ClassificationBase(nn.Module):
    def training_step(self, batch):
        #inputs, classes = batch
        images, targets = batch
        images = images.type(torch.FloatTensor) # Uncomment for BreastCancer ClassfierBase class
        #images = torch.reshape(images.type(torch.DoubleTensor), (len(images), 1))
        targets = torch.reshape(targets.type(torch.FloatTensor), (len(targets), 1))
        out = self(images)
        loss = F.binary_cross_entropy(out, targets)
        return loss

    def validation_step(self, batch):
        images, targets = batch
        images = images.type(torch.FloatTensor) # Uncomment for BreastCancer ClassfierBase class
        #images = torch.reshape(images.type(torch.DoubleTensor), (len(images), 1))
        #print(images)
        targets = torch.reshape(targets.type(torch.FloatTensor), (len(targets), 1))
        #print(targets)
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))

### Functions to evaluate

In [199]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [200]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    #writer = SummaryWriter()  # Create a SummaryWriter instance

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []  # learning rate
        step = 0  # Initialize the step counter
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Write the training loss to TensorBoard with unique step for each batch
            writer.add_scalar('Training Batch Loss', loss, step)
            step += 1  # Increment the step counter

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Write the training loss and learning rate to TensorBoard
        writer.add_scalar('Training Loss', torch.stack(train_losses).mean().item(), epoch)
        writer.add_scalar('Learning Rate', lrs[-1], epoch)

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

    return history

def plot_scores(history):
    scores = [x['val_score'] for x in history]
    plt.plot(scores, '-x')
    plt.xlabel('epoch')
    plt.ylabel('score')
    plt.title('F1 score vs. No. of epochs')
    plt.show()
    #plt.savefig("DNN_scores_no_augmentation")

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.show()
    #plt.savefig("DNN_losses_no_augmentation")

def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.')
    plt.show()
    #plt.savefig("DNN_lrs_no_augmentation")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Load device


In [201]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda') #REQUIRES CHANGING THE TORCH.FLOATTENSOR TO TORCH.CUDA.FLOATTENSOR
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

device(type='cpu')

### Load data

In [202]:
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(val_loader, device)

#3. Baseline accuracy

## 3.1 FC model

TODO: Input_size in forward is hardcoded as grid size

In [203]:
class FCNN(ClassificationBase):
    def __init__(self, input_size):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.Dropout(p=0.15),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Dropout(p=0.1),
            nn.ReLU(),
            nn.Linear(512, 1)
        )
    #HERE THE MODEL PERFORMS A FORWARD PASS --> OUTPUT/PREDICTION
    def forward(self, xb, input_size=200*200):
        xb = xb.reshape(-1,input_size)
        xb = xb.to(torch.float32)  # Convert input to float32 data type
        out = self.linear(xb)
        #out = out.to(torch.float32) # Leave as comment for DNN 1
        return torch.sigmoid(out)


## 3.2 Run model
Saves output in tensorboard

### 3.2.1 Initialize

In [204]:
writer = SummaryWriter()
model = to_device(FCNN(input_size=200*200), device) #, flat_shape=9801), device) #DONT WANT THIS TO BE AN INPUT!
epochs = 100
max_lr = 0.01
opt_func = torch.optim.Adam

### 3.2.2 Run classifier

In [205]:
history = [evaluate(model, val_dl)]
history

[{'val_loss': 0.6887596845626831, 'val_score': 0.8333332538604736}]

### 3.2.3 Train

In [206]:
# start_time = time.time()
# history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, opt_func=opt_func)
# train_time = time.time() - start_time
# total_train_time = time.time() - start_time
# print("Total training time =", total_train_time)

### 3.2.3 Evaluate

Adjust name and description as needed

In [207]:
# writer.flush
# writer.close()

In [208]:
# !yes|tensorboard dev upload --logdir /content/runs/ --name "DNN() Fully Connected " --description "Fully connected to evaluate baseline accuracy"


# 4. GridSearch with fully connected layers

In [209]:
# from itertools import product

# # Define the hyperparameter values to explore
# epochs_values = [1, 2, 3]
# max_lr_values = [0.01, 0.001]

# best_score = 0.0
# best_epochs = 0
# best_max_lr = 0.0

# product(epochs_values, max_lr_values)

In [210]:
# for epochs, max_lr in product(epochs_values, max_lr_values):

#     # Create a unique tag for each run based on the hyperparameters
#     tag = f"epochs_{epochs}_max_lr_{max_lr}"

#     # Create a SummaryWriter instance for each run
#     writer = SummaryWriter(log_dir=f"runs/{tag}")

#     # Create a new instance of the model for each combination of hyperparameters
#     model = to_device(DNN(input_size=200*200), device) #, flat_shape=9801), device) #DONT WANT THIS TO BE AN INPUT!

#     # Train the model and evaluate its performance
#     history = [evaluate(model, val_dl)]
#     history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, opt_func=opt_func)

#     # Calculate the validation score
#     final_score = history[-1]['val_score']

#     # Check if the current combination is the best
#     if final_score > best_score:
#         print("Better parameters found, updating.")
#         best_score = final_score
#         best_epochs = epochs
#         best_max_lr = max_lr

#     # Print the validation score for the current combination
#     print(f"Epochs: {epochs}, Max LR: {max_lr}, Validation Score: {final_score}")

#     # Close the writer for each run
#     writer.close()


# # Print the best hyperparameters
# print("Best Hyperparameters:")
# print("Epochs:", best_epochs)
# print("Max LR:", best_max_lr)

In [211]:
#!yes|tensorboard dev upload --logdir /content/runs/ --name "DNN() GRIDSearch " --description "Fully connected to evaluate baseline accuracy"

# 5. GridSearch with convolutional layers
Right now a limitation is the changing outputs/mismatch between convolutional layer sizes. Have just changed it to make it consistent.
I.e. Size of layer 1 == size of layer 2.

In [212]:
class CNNGridSearch(ClassificationBase):
    def __init__(
        self,
        input_shape,
        conv1_filters,
        conv1_kernel_size,
        conv2_filters,
        conv2_kernel_size,
        maxpool_kernel_size,
        dropout,
        fc1_nodes,
        #use_second_dense,
        fc2_nodes,
        #use_third_dense,
        fc3_nodes,
        use_second_conv_block,
        flat_features,
        conv_stride,
    ):
        super().__init__()
        channels, height, width = input_shape
        ## First convolutional layer
        # "Uses three filters to scan each row of the CyTOF data. This layer extracts relevant information from the cell marker profile of each cell." Is this grid AxBxC? Fix in inputShape[X]. Filter size = 1 x B
        # We want to measure C markers.
        # How many output markers
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=conv1_filters, kernel_size=conv1_kernel_size) #(1,A)? - THE NUMBER OF NODES IN THE INPUT VECTOR. OR JUST KERNEL SIZE = 3?
        self.bn1 = torch.nn.BatchNorm2d(conv1_filters)
        self.act1 = nn.ReLU()

        ## Second (optional) convolutional layer
        self.use_second_conv_block = use_second_conv_block
        if self.use_second_conv_block:
            self.conv2 = nn.Conv2d(in_channels = conv1_filters, out_channels=conv2_filters, kernel_size=conv2_kernel_size)
            self.bn2 = torch.nn.BatchNorm2d(conv2_filters)
            self.act2 = nn.ReLU()

            ## Pooling layer
            # "The pooling layers averages the outputs of the second convolution layer. The purpose is to aggregate the cell level information into sample-level information.""
            self.pool = nn.MaxPool2d(kernel_size=maxpool_kernel_size, stride=conv_stride)
            self.flat = nn.Flatten()
        else:

          ## Pooling layer
          # "The pooling layers averages the outputs of the first convolution layer. The purpose is to aggregate the cell level information into sample-level information.""
          self.pool = nn.MaxPool2d(kernel_size=maxpool_kernel_size, stride=conv_stride)
          self.flat = nn.Flatten()

        ## Dense layers
        # "The dense layer further extracts information from the pooling layer."
        self.fc1 = nn.Linear(in_features=flat_features, out_features=fc1_nodes) #flat_features = 10
        self.bn3 = torch.nn.BatchNorm1d(fc1_nodes)
        self.act3 = nn.ReLU()
        self.do1 = nn.Dropout(p=dropout)

        # Second dense layer
        self.fc2 = nn.Linear(in_features=fc1_nodes, out_features=fc2_nodes)
        self.bn4 = torch.nn.BatchNorm1d(fc2_nodes)
        self.act4 = nn.ReLU()
        self.do2 = nn.Dropout(p=dropout)

        # Third dense layer
        self.fc3 = nn.Linear(in_features=fc2_nodes, out_features=fc3_nodes)
        self.bn5 = torch.nn.BatchNorm1d(fc3_nodes)
        self.act5 = nn.ReLU()
        self.do3 = nn.Dropout(p=dropout)

        ## Output layer
        # "Uses logistic regression to report the probability of AD for each sample."
        self.fc5 = nn.Linear(in_features=fc3_nodes, out_features=1)
        self.bn6 = nn.BatchNorm1d(1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = x.float()
        x = x.unsqueeze(1)
        #print("Input dimensions", x.shape)
        x = self.act1(self.bn1(self.conv1(x)))
        #print("Input dimensions 1st conv layer", x.shape)
        if self.use_second_conv_block:
            x = self.act2(self.bn2(self.conv2(x)))
            x = self.flat(self.pool(x))
            #print("Input dimensions flat features conv2", x.shape)
        else:
            x = self.flat(self.pool(x))
            #print("Input dimensions flat features conv1", x.shape)

        x = self.do1(self.act3(self.bn3(self.fc1(x))))
        x = self.do2(self.act4(self.bn4(self.fc2(x))))
        x = self.do3(self.act5(self.bn5(self.fc3(x))))
        out = self.bn6(self.fc5(x))
        return self.sigmoid(out)


In [213]:
from itertools import product
input_shape = [1,200,200]
# Define the hyperparameter values to explore
epochs_values = [50, 100]
max_lr_values = [0.01, 0.001]
input_shape = [1,200,200]
#conv1_filters = [1,3, 5]
#conv1_kernel_size = [2, 5, 9]
conv_filters = [1, 3, 5]
conv_kernel_size = [2, 5, 9]
#conv_stride = [2]
#maxpool_kernel_size = [2]
conv2_filters = [1,3, 5]
conv2_kernel_size = [2, 5, 9]
#dropout = [0.1]
#fc1_nodes = [2048]
#fc2_nodes = [512]
#fc3_nodes = [128]
use_second_conv_block = [True, False]

best_score = 0.0
best_epochs = 0
best_max_lr = 0.0

## Function to calculate the number of flat features required in the dense layers.
Note that filter sizes must be square (W = H)

In [214]:
def calculate_flat_features(input_shape, conv_kernel_size, conv_stride, use_second_conv_block):
    channels, height, width = input_shape

    # First convolutional layer
    conv1_output = ((height - conv_kernel_size)/conv_stride)+1
    print(conv1_output)
    if use_second_conv_block:
        # Flat features size after pooling
        conv2_output = ((conv1_output - conv_kernel_size)/conv_stride)+1
        flat_features = conv2_output**2
    else:
        # Flat features size after pooling
        flat_features = conv1_output**2

    return int(flat_features)

In [215]:
calculate_flat_features(input_shape, 1, 1, True)

200.0


40000

Run models

In [ ]:
for epochs, max_lr, conv_filter, conv_kernel, use_second in product(epochs_values, max_lr_values, conv_filters, conv_kernel_size, use_second_conv_block):
#for epochs, max_lr, conv1_filter, conv1_kernel, conv2_filter, conv2_kernel, use_second in product(epochs_values, max_lr_values, conv1_filters, conv1_kernel_size, conv2_filters, conv2_kernel_size, use_second_conv_block):

    # Create a unique tag for each run based on the hyperparameters
    tag = f"{epochs}_{max_lr}_{conv_filter}_{conv_kernel}_{use_second}"
    print("Running", tag)

    # Create a SummaryWriter instance for each run
    writer = SummaryWriter(log_dir=f"runs/gridCNN1406/{tag}")

    # Determine the number of flat features
    # flat_feature = calculate_flat_features(input_shape=input_shape,
    #                                        #conv1_filters=conv1_filter,
    #                                        conv1_kernel_size=conv1_kernel,
    #                                        #conv2_filters=conv2_filter,
    #                                        conv2_kernel_size=conv2_kernel,
    #                                        conv_stride=2,
    #                                        use_second_conv_block=use_second)
    # print(flat_feature)
    # Create a new instance of the model for each combination of hyperparameters

    print("Device:", device)
    model = to_device(CNNGridSearch(input_shape=input_shape,
                                    conv1_filters=conv_filter,
                                    conv1_kernel_size=conv_kernel,
                                    maxpool_kernel_size=2,
                                    conv2_filters=conv_filter,
                                    conv2_kernel_size=conv_kernel,
                                    conv_stride = 2,
                                    use_second_conv_block=use_second,
                                    dropout=0.1,
                                    fc1_nodes=2048,
                                    fc2_nodes=512,
                                    fc3_nodes=128,
                                    flat_features=9801), device)

    # Train the model and evaluate its performance
    history = [evaluate(model, val_dl)]
    history += fit_one_cycle(epochs=epochs, max_lr=max_lr, model=model, train_loader=train_dl, val_loader=val_dl, opt_func=opt_func)

    # Calculate the validation score
    final_score = history[-1]['val_score']

    # Check if the current combination is the best
    if final_score > best_score:
        print("Better parameters found, updating.")
        best_score = final_score
        best_epochs = epochs
        best_max_lr = max_lr

    # Print the validation score for the current combination
    print(f"Epochs: {epochs}, Max LR: {max_lr}, Validation Score: {final_score}")

    # Close the writer for each run
    writer.close()


# Print the best hyperparameters
print("Best Hyperparameters:")
print("Epochs:", best_epochs)
print("Max LR:", best_max_lr)

Running 50_0.01_1_2_True
Device: cpu


100%|██████████| 21/21 [00:13<00:00,  1.53it/s]


Epoch [0], last_lr: 0.0005, train_loss: 0.7690, val_loss: 0.7184, val_score: 0.0000


100%|██████████| 21/21 [00:13<00:00,  1.56it/s]


Epoch [1], last_lr: 0.0008, train_loss: 0.6185, val_loss: 0.8125, val_score: 0.0000


...

100%|██████████| 21/21 [00:12<00:00,  1.68it/s]


Epoch [48], last_lr: 0.0000, train_loss: 0.0953, val_loss: 0.0979, val_score: 0.8351


100%|██████████| 21/21 [00:12<00:00,  1.65it/s]


Epoch [49], last_lr: 0.0000, train_loss: 0.1007, val_loss: 0.0957, val_score: 0.8351
Better parameters found, updating.
Epochs: 50, Max LR: 0.01, Validation Score: 0.835106372833252
Running 50_0.01_1_2_False
Device: cpu


100%|██████████| 21/21 [00:13<00:00,  1.59it/s]


In [ ]:
!yes|tensorboard dev upload --logdir /content/runs/gridCNN1406/ --name "CNN() GRIDSearch Try" --description "Keeping dimensions the same for number of conv1 and conv2 channels"